In [1]:
import numpy as np

import pandas as pd 
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

<font size="6">Reading Wikipedia Data and converting the data into a Data Frame</font>

In [2]:
from bs4 import BeautifulSoup
#Reading HTMl page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response=requests.get(url)
html = response.text
page = BeautifulSoup(html,'lxml')

In [84]:
table_rows=page.find_all('tr')
#table_rows

In [4]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

In [5]:
df = pd.DataFrame(l) 
df.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,M1A,Not assigned,Not assigned\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,M2A,Not assigned,Not assigned\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,M3A,North York,Parkwoods\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,M4A,North York,Victoria Village\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [6]:
df.rename(columns={0: 'PostalCode', 1: 'Borough', 2: 'Neighbourhood'}, inplace=True)

In [7]:
df.head()

,PostalCode,Borough,Neighbourhood,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,M1A,Not assigned,Not assigned\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,M2A,Not assigned,Not assigned\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,M3A,North York,Parkwoods\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,M4A,North York,Victoria Village\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


<font size="6">Removing unwanted columns and rows and further formatting the Data Frame</font>

In [8]:
df.drop(df.iloc[:, 3:31], axis=1, inplace=True)

In [9]:
df.head()

,PostalCode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


<font size="6">Removing rows with incomplete data centered on column 'Borough' and formatting</font>

In [10]:
df.replace('Not assigned',np.NaN, inplace=True)

In [11]:
df.dropna(subset=['Borough'],inplace=True)

In [12]:
df.reset_index(drop=True,inplace=True)

In [13]:
df['Neighbourhood']=df['Neighbourhood'].str.slice(start=0,stop=-1)

In [14]:
df.replace('Not assigned',np.NaN, inplace=True)

In [15]:
df=df[:-4]

<font size="6">Replacing null values in Neighbourhood Column with values from Borough column</font>

In [16]:
df['Neighbourhood'].fillna(value=df['Borough'],inplace=True)

In [17]:
columns=['PostalCode','Borough','Neighbourhood']

In [18]:
df_temp=pd.DataFrame(columns=columns)
df_new=pd.DataFrame(columns=columns)
df_new

,PostalCode,Borough,Neighbourhood


<font size="6">Replacing more than one values in Neighbourhood against the same value in Borough as a comma separated list</font>

In [19]:
codes=df.PostalCode.unique()

In [20]:
for i in codes:
    df_temp=df[df['PostalCode']==i]
    str1=""
    borough=""
    for index, row in df_temp.iterrows():
        str1=str1+df_temp['Neighbourhood'].get_value(index) + ', '
        borough=df_temp['Borough'].get_value(index)
    df_new = df_new.append({'PostalCode': i, 'Borough': borough, 'Neighbourhood': str1}, ignore_index=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [21]:
df_new['Neighbourhood']=df_new['Neighbourhood'].str.slice(start=0,stop=-2)

In [22]:
df_new.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [23]:
df_new.shape

(103, 3)

In [24]:
# @hidden_cell
CLIENT_ID = 'V2OTMODD3BRUJCC2OTYC4SDGKNFKMBOZHHYPIMDERZXK1ZQP' # your Foursquare ID
CLIENT_SECRET = 'JTYZ25PXWTOMMEMKKBMYYNIW1IZOEFVIXLIWEQENPJATRMVX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

<font size="6">Getting the Latitude-Longitude data</font>

In [25]:
#import geocoder

#for row in df_new['PostalCode']:
#   location = geocoder.google('{}, Toronto, Ontario'.format(row))
#    print(location)

In [26]:
#https://cocl.us/Geospatial_data
df_coords=pd.read_csv('https://cocl.us/Geospatial_data')

In [27]:
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
df_coords.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

In [29]:
df_coords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<font size="6">Merging the newly created Data Frames</font>

In [31]:
df_merged=pd.merge(df_coords, df_new, on='PostalCode')

In [32]:
df_merged.reset_index(drop=True,inplace=True)
df_merged

,PostalCode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
5,M1J,43.744734,-79.239476,Scarborough,Scarborough Village
6,M1K,43.727929,-79.262029,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,43.711112,-79.284577,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,43.716316,-79.239476,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,43.692657,-79.264848,Scarborough,"Birch Cliff, Cliffside West"


In [33]:
df_merged.shape

(103, 5)

<font size="6">Considering only those columns with the name 'Toronto'</font>

In [34]:
df_filtered=df_merged[df_merged['Borough'].str.contains("Toronto")]
df_filtered.head()

,PostalCode,Latitude,Longitude,Borough,Neighbourhood
37,M4E,43.676357,-79.293031,East Toronto,The Beaches
41,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale"
42,M4L,43.668999,-79.315572,East Toronto,"The Beaches West, India Bazaar"
43,M4M,43.659526,-79.340923,East Toronto,Studio District
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park


In [35]:
df_filtered.shape

(39, 5)

<font size="6">Exploring neighbourhoods in Toronto</font>

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
LIMIT=30
Toronto_Venues = getNearbyVenues(names=df_filtered['Neighbourhood'],
                                  latitudes=df_filtered['Latitude'],
                                  longitudes=df_filtered['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [44]:
Toronto_Venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
3,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [45]:
Toronto_Venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
"Chinatown, Grange Park, Kensington Market",30,30,30,30,30,30
Christie,18,18,18,18,18,18


In [53]:
print('There are {} uniques categories.'.format(len(Toronto_Venues['Venue Category'].unique())))

There are 190 uniques categories.


<font size="6">Analyze each neighbourhood</font>

In [46]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_Venues[['Venue Category']], prefix="", prefix_sep="")

#add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_Venues['Neighbourhood'] 

# move neighborhood column to the first column

fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])

Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head(2)

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
Toronto_onehot.shape

(864, 191)

In [48]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.033333,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000


In [49]:
Toronto_grouped.shape

(39, 191)

In [50]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                venue  freq
0          Steakhouse  0.10
1  Seafood Restaurant  0.07
2    Asian Restaurant  0.07
3               Hotel  0.07
4               Plaza  0.03


----Berczy Park----
          venue  freq
0      Beer Bar  0.07
1        Bakery  0.07
2          Café  0.07
3  Cocktail Bar  0.07
4   Coffee Shop  0.07


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1       Nightclub  0.09
2     Coffee Shop  0.09
3            Café  0.09
4             Gym  0.04


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0       Pizza Place  0.06
1  Recording Studio  0.06
2           Butcher  0.06
3        Smoke Shop  0.06
4           Brewery  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3        

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Asian Restaurant,Hotel,Seafood Restaurant,Opera House,Monument / Landmark,Coffee Shop,Pizza Place,Bar,Lounge
1,Berczy Park,Farmers Market,Seafood Restaurant,Beer Bar,Coffee Shop,Cocktail Bar,Café,Bakery,Basketball Stadium,Fish Market,Bistro
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Nightclub,Coffee Shop,Café,Bakery,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Smoke Shop,Fast Food Restaurant,Park,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Light Rail Station,Brewery
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Airport,Rental Car Location,Airport Food Court,Bar,Boat or Ferry,Harbor / Marina,Sculpture Garden


<font size="6">Cluster Neighbourhoods</font>

In [59]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [63]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_filtered

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = df_filtered.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Latitude,Longitude,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,43.676357,-79.293031,East Toronto,The Beaches,0,Neighborhood,Other Great Outdoors,Coffee Shop,Health Food Store,Trail,Pub,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
41,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale",0,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Spa,Pub,Pizza Place,Juice Bar,Furniture / Home Store,Fruit & Vegetable Store
42,M4L,43.668999,-79.315572,East Toronto,"The Beaches West, India Bazaar",0,Sandwich Place,Food & Drink Shop,Burger Joint,Brewery,Liquor Store,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop,Steakhouse,Ice Cream Shop
43,M4M,43.659526,-79.340923,East Toronto,Studio District,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Pet Store,Seafood Restaurant,Brewery,Sandwich Place,Cheese Shop
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park,1,Park,Bus Line,Swim School,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [83]:
# create map

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<font size="6">Examining Clusters</font>

<font size="5">Cluster 1</font>

In [70]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,43.676357,0,Neighborhood,Other Great Outdoors,Coffee Shop,Health Food Store,Trail,Pub,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
41,43.679557,0,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Spa,Pub,Pizza Place,Juice Bar,Furniture / Home Store,Fruit & Vegetable Store
42,43.668999,0,Sandwich Place,Food & Drink Shop,Burger Joint,Brewery,Liquor Store,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop,Steakhouse,Ice Cream Shop
43,43.659526,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Pet Store,Seafood Restaurant,Brewery,Sandwich Place,Cheese Shop
45,43.712751,0,Pizza Place,Hotel,Sandwich Place,Food & Drink Shop,Breakfast Spot,Department Store,Park,Gym,Cosmetics Shop,Coworking Space
46,43.715383,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Park
47,43.704324,0,Dessert Shop,Sushi Restaurant,Sandwich Place,Café,Coffee Shop,Italian Restaurant,Gym,Pizza Place,Pharmacy,Brewery
49,43.686412,0,Coffee Shop,Pub,Supermarket,Light Rail Station,Vietnamese Restaurant,Liquor Store,Sushi Restaurant,American Restaurant,Pizza Place,Restaurant
51,43.667967,0,Coffee Shop,Italian Restaurant,Restaurant,Café,Pub,Jewelry Store,Sandwich Place,Butcher,Caribbean Restaurant,Liquor Store
52,43.665860,0,Gay Bar,Burger Joint,Park,Dance Studio,Bubble Tea Shop,Salon / Barbershop,Restaurant,Ramen Restaurant,Pub,Pizza Place


<font size="5">Cluster 2</font>

In [71]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,43.72802,1,Park,Bus Line,Swim School,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


<font size="5">Cluster 3</font>

In [72]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,43.711695,2,Pool,Garden,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


<font size="5">Cluster 4</font>

In [73]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,43.689574,3,Tennis Court,Playground,Summer Camp,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


<font size="5">Cluster 5</font>

In [74]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,43.679563,4,Park,Playground,Trail,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
64,43.696948,4,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
